In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import shutil
from pathlib import Path
from typing import List

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import numpy as np
from ipywidgets import interact, Layout
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm

from visualisation import render_plot
from exporter import export_obj
from depthmap import Depthmap

In [ ]:
TOOLKIT_DIR = Path(os.getcwd()).absolute()
TOOLKIT_DIR

In [ ]:
depthmap_dir = TOOLKIT_DIR / 'tests' / 'huawei_p40pro'
depthmap_fname = 'depth_dog_1622182020448_100_282.depth'
rgb_fname = 'rgb_dog_1622182020448_100_282.jpg'
calibration_file = TOOLKIT_DIR / 'tests' / 'huawei_p40pro' / 'camera_calibration.txt'

In [ ]:
def check_depthmap_and_rgb_frames_are_in_same_order(depthmap_fnames: List[str], rgb_fnames: List[str]):
    for depthmap_fname, rgb_fname in zip(depthmap_fnames, rgb_fnames):
        assert depthmap_fname.startswith('depth')
        assert rgb_fname.startswith('rgb')
        assert depthmap_fname.endswith('.depth')
        assert rgb_fname.endswith('.jpg')

        _depthmap_fname = depthmap_fname.replace('depth_', '', 1)
        _rgb_fname = rgb_fname.replace('rgb_', '', 1)
        _depthmap_fname = _depthmap_fname.split('.')[0]
        _rgb_fname = _rgb_fname.split('.')[0]

        assert _depthmap_fname == _rgb_fname, (_depthmap_fname, _rgb_fname)

In [ ]:
depthmap_fnames = sorted(map(lambda x: x.name, depthmap_dir.rglob('*.depth')))
rgb_fnames = sorted(map(lambda x: x.name, depthmap_dir.rglob('*.jpg')))
check_depthmap_and_rgb_frames_are_in_same_order(depthmap_fnames, rgb_fnames)

In [ ]:
dmaps, dmaps_visualizations = [], []
for depthmap_fname, rgb_fname in tqdm(list(zip(depthmap_fnames, rgb_fnames))):
    dmap = Depthmap.create_from_zip(str(depthmap_dir), str(depthmap_fname), str(rgb_fname), str(calibration_file))
    dmaps.append(dmap)
    dmaps_visualizations.append(render_plot(dmap))  # takes 5 seconds each time

In [ ]:
# Easy visualization

def trigger_visualization(artifact_idx):
    plt.imshow(dmaps_visualizations[artifact_idx])
    plt.show()

slider = widgets.IntSlider(description='artifact_idx', value=0, min=0, max=2);
plot_out = widgets.interactive_output(trigger_visualization, {'artifact_idx': slider});
display(widgets.VBox([slider, plot_out], layout=Layout(width='50%')))

In [ ]:
# Advanced visualization

class Viz:
    def __init__(self, dmaps: List[Depthmap], dmaps_visualizations: List[np.array]):
        assert isinstance(dmaps_visualizations, List)
        assert len(dmaps) == len(dmaps)
        self.dmaps = dmaps
        self.dmaps_visualizations = dmaps_visualizations
        self.num_artifacts = len(dmaps_visualizations)
        self.setup_ui()
        
    def setup_ui(self):
        self.slider = widgets.IntSlider(description='artifact_idx', value=0, min=0, max=self.num_artifacts - 1)
        self.plot_out = widgets.interactive_output(self.trigger_visualization, {'artifact_idx': self.slider})
        self.export_obj_button = widgets.Button(description="Export OBJ!")
        self.export_obj_button.on_click(self.on_export_obj_clicked)
        self.message_textbox = widgets.Text(description='')

        display(self.slider)
        display(self.plot_out);
        display(widgets.HBox([self.export_obj_button, self.message_textbox], layout=Layout(width='50%')))
        
    def trigger_visualization(self, artifact_idx):
        plt.imshow(self.dmaps_visualizations[artifact_idx])
        plt.show()

    def on_export_obj_clicked(self, b):
        artifact_idx = self.slider.value
        self.message_textbox.value = str(f"Exporting artifact with index '{artifact_idx}'")
        dmap = dmaps[artifact_idx]
        floor = dmap.get_floor_level()
        fname = f'output{artifact_idx}.obj'
        export_obj(f'export/{fname}', dmap, floor, triangulate=True)
        self.message_textbox.value = str(f"Exported artifact with index '{artifact_idx}'")

In [ ]:
print("Use the slider to navigate between artifacts, click the button to export the current image")
viz = Viz(dmaps=dmaps, dmaps_visualizations=dmaps_visualizations)